In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import os
import time
import json
import openai
from dotenv import load_dotenv

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

from method.models import generate_random_values, generate_static_values, generate_llm_values
from method.ours.utils import create_driver, get_xpath, interact_with_input
from method.ours.history import HistoryTable
from method.ours.feedback import get_global_feedback

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

HEADLESS = False
URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
driver = create_driver(HEADLESS)
driver.get(URL)

[WDM] - Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.29M/8.29M [00:00<00:00, 20.2MB/s]


In [8]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')
form = driver.find_elements(By.TAG_NAME, 'form')[1]
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [9]:
inputs

[<selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e457871bd", element="327716FAECE1EDF4152D00F598DD68EF_element_33")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e457871bd", element="327716FAECE1EDF4152D00F598DD68EF_element_34")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e457871bd", element="327716FAECE1EDF4152D00F598DD68EF_element_35")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e457871bd", element="327716FAECE1EDF4152D00F598DD68EF_element_37")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e457871bd", element="327716FAECE1EDF4152D00F598DD68EF_element_39")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e457871bd", element="327716FAECE1EDF4152D00F598DD68EF_element_41")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ea7228f7e4939b2478511e4578

In [24]:
generate_llm_values(form.get_attribute('outerHTML'), openai.api_key)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate testing values for a web form. When given a web form, you need to generate values that can pass and fail when tried on the form. Your response should be in the following format:\n```\n{\n[field_identifier]: {\npassing: <a base passing value for the field>,\nfailing: <a list of failing values for the field>\n}\n}\n```\n"
  },
  {
    "role": "user",
    "content": "<form novalidate=\"\" autocomplete=\"off\" class=\"bkmg-tab-content-container bkmg-flights-tab-container ng-dirty ng-invalid ng-touched\"><fieldset><legend class=\"visually-hidden\"> Flights </legend><div class=\"bkmg-flights-tab-grid\"><fieldset class=\"bkmg-flights-tab-trip-type-selector mobile-padding\"><legend class=\"visually-hidden\"> Trip type </legend><abc-radio-group formcontrolname=\"tripType\" class=\"ng-untouched ng-pristine ng-valid abc-form-element\"><div class=\"abc-form-element-container abc-radio-group-container\"><div class=\"ab

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "```\n{\n\"tripType\": {\npassing: \"R\",\nfailing: [\"A\", \"B\", \"C\", \"D\", \"E\", \"F\", \"G\", \"H\", \"I\", \"J\", \"K\", \"L\", \"N\", \"P\", \"Q\", \"S\", \"T\", \"U\", \"V\", \"W\", \"X\", \"Y\", \"Z\", \"\", \" \", \"1\", \"2\", \"3\", \"4\", \"5\", \"6\", \"7\", \"8\", \"9\", \"0\"]\n},\n\"searchType\": {\npassing: \"cash\",\nfailing: [\"credit\", \"debit\", \"points\", \"\", \" \", \"1\", \"2\", \"3\", \"4\", \"5\", \"6\", \"7\", \"8\", \"9\", \"0\"]\n},\n\"origin\": {\npassing: \"Vancouver\",\nfailing: [\"\", \" \", \"123\", \"abc\", \"Vancouv\", \"Vancou\", \"Vanco\", \"Vanc\", \"Van\", \"Va\", \"V\"]\n},\n\"destination\": {\npassing: \"Toronto\",\nfailing: [\"\", \" \", \"123\", \"abc\", \"Toront\", \"Toron\", \"Toro\", \"Tor\", \"To\", \"T\"]\n},\n\"travelDates\": {\npassing: \"12/12/2022\",\nfailing: [\"\", \" \", \"12/12/22\", \"12-12-2022\", \"1

AttributeError: 'str' object has no attribute 'read'